In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import random

In [4]:
# Parameters
N_AGENTS = 100
BAR_CAPACITY = 60
WEEKS = 100
SEQUENCE_LENGTH = 5

# Generate dummy attendance history
def generate_initial_attendance(weeks=SEQUENCE_LENGTH):
    return [random.randint(40, 70) for _ in range(weeks)]

# Build LSTM predictor
def build_lstm_model():
    model = Sequential([
        LSTM(10, input_shape=(SEQUENCE_LENGTH, 1), return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Simple Q-learning Agent
class Agent:
    def __init__(self):
        self.q_table = np.zeros((2,))  # 0: stay home, 1: go
        self.alpha = 0.1
        self.gamma = 0.95
        self.epsilon = 0.2
        self.lstm = build_lstm_model()
        self.past_attendance = generate_initial_attendance()
        # Train the LSTM model with the dummy data
        X = np.array([self.past_attendance[i:i+SEQUENCE_LENGTH] for i in range(len(self.past_attendance) - SEQUENCE_LENGTH)])
        y = np.array(self.past_attendance[SEQUENCE_LENGTH:])
        if len(X) > 0:
            self.lstm.fit(X.reshape(-1, SEQUENCE_LENGTH, 1), y, epochs=20, verbose=0)

    def predict_attendance(self):
        sequence = np.array(self.past_attendance[-SEQUENCE_LENGTH:]).reshape(1, SEQUENCE_LENGTH, 1)
        prediction = self.lstm.predict(sequence, verbose=0)[0][0]
        return prediction

    def decide(self, predicted_attendance):
        if random.random() < self.epsilon:
            return random.choice([0, 1])
        else:
            return np.argmax(self.q_table)

    def update_q(self, action, reward):
        self.q_table[action] = self.q_table[action] + self.alpha * (reward + self.gamma * np.max(self.q_table) - self.q_table[action])

    def update_attendance_history(self, value):
        self.past_attendance.append(value)
        if len(self.past_attendance) > SEQUENCE_LENGTH + 1:
            self.past_attendance.pop(0)

# Initialize agents
agents = [Agent() for _ in range(N_AGENTS)]

# Simulation
history = generate_initial_attendance(weeks=SEQUENCE_LENGTH)
for week in range(WEEKS):
    decisions = []
    predictions = []
    for agent in agents:
        pred = agent.predict_attendance()
        predictions.append(pred)
        action = agent.decide(pred)
        decisions.append(action)
    
    attendance = sum(decisions)
    history.append(attendance)
    history = history[-(SEQUENCE_LENGTH + 1):]

    for i, agent in enumerate(agents):
        reward = 1 if (decisions[i] == 1 and attendance <= BAR_CAPACITY) else -1 if decisions[i] == 1 else 0.5
        agent.update_q(decisions[i], reward)
        agent.update_attendance_history(attendance)


C:\Users\k3n\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
print("BAR_CAPACITY:", BAR_CAPACITY)
print("N_AGENTS:", N_AGENTS)
print("SEQUENCE_LENGTH:", SEQUENCE_LENGTH)
print("WEEKS:", WEEKS)
print("Current Week:", week)
print("Attendance:", attendance)
print("Decisions:", decisions)
print("Predictions:", predictions)
print("History:", history)
print("Reward:", reward)

BAR_CAPACITY: 60
N_AGENTS: 100
SEQUENCE_LENGTH: 5
WEEKS: 100
Current Week: 99
Attendance: 31
Decisions: [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
Predictions: [-0.040037114, -0.80520296, -0.4072002, 0.75555164, 0.29798466, -0.11574817, -0.4437642, 0.6173918, -1.2909397, -0.59182274, -1.2068125, 0.14483353, 1.1339676, 0.6434838, -1.9307551, 0.12221027, 0.29612422, -0.5775117, 0.0979374, -0.5286367, -1.0675871, -0.5562646, 0.5063758, -0.43042237, 0.60359734, 0.3607213, -0.6842465, 0.20143822, 0.6790423, 0.85435593, 0.6081389, 0.40812677, -0.1602152, 0.19909333, 0.51742697, 1.5107089, 0.2864718, -0.06613885, 0.21032326, -1.3621294, 0.9226646, 0.10392186, 0.20023642, -0.20538048, 0.32023907, -0.08152153, 0.3980237, -0.063250095, 0.2375